Feature Engineering ใน Notebook นี้จัดทำขึ้นเพื่อ clean feature ทั้ง Numerical และ Categorical โดยมีทดลองจัดการกับ Value เหล่านี้ 2 แบบ
* แบบที่ 1 :
    * Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และตัด row ที่มี missing value ออก
    * Numerical : นำ Row ที่เป็น Outlier และ Missing Value ออก
* แบบที่ 2 :
    * Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และเติม Missing value ด้วย Mode
    * Numerical : เติม Missing value ด้วย median และ cap ค่า outlier

In [1]:
from src.clean_data import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
def remove_outliers(df, numerical_columns):
    """ลบ outlier ออกจากคอลัมน์ที่กำหนดโดยใช้ IQR"""
    df_clean = df.copy()  # สร้างสำเนา DataFrame เพื่อลบแถว
    
    for column in numerical_columns:
        Q1 = df_clean[column].quantile(0.25)  # ค่ามัธยฐานล่าง (25th percentile)
        Q3 = df_clean[column].quantile(0.75)  # ค่ามัธยฐานบน (75th percentile)
        IQR = Q3 - Q1  # ค่าความกว้างระหว่างควอไทล์
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # กรองแถวที่ไม่มี outlier
        df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]
    
    return df_clean

In [3]:
def scale_data(data,scale_by='standard_scaler',target_col = 'มูลค่าความเสียหาย'):
    if scale_by == 'standard_scaler':
        # ใช้ StandardScaler
        scaler = StandardScaler()
    elif scale_by == 'min_max':
        scaler = MinMaxScaler()
        
    features = data.drop(target_col,axis=1)
    scaled_features = scaler.fit_transform(features)

    # สร้าง DataFrame ใหม่ที่สเกลแล้ว
    df_scaled = pd.DataFrame(scaled_features, columns=features.columns)

    # นำคอลัมน์ target กลับมา
    df_scaled[target_col] = data[target_col]
    return df_scaled

In [5]:
raw_data = pd.read_csv('./data/accident2024 + $Damage.csv')
target = 'มูลค่าความเสียหาย'

# นำ Column ที่ไม่เกี่ยวข้องออก
irrelevant_column = ['ปีที่เกิดเหตุ', 'วันที่เกิดเหตุ', 'เวลา', 'วันที่รายงาน',
                     'เวลาที่รายงาน', 'ACC_CODE', 'หน่วยงาน', 'สายทางหน่วยงาน', 
                     'รหัสสายทาง','สายทาง', 'KM']
df = raw_data.drop(irrelevant_column,axis=1)


In [6]:
# ระบุ column ที่เป็น categorical และ numerical
categorical_feature = ['บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน',
               'ลักษณะการเกิดอุบัติเ', 'สภาพอากาศ']
numerical_feature = ['รถจักรยานยนต์', 'รถสามล้อเครื่อง',
       'รถยนต์นั่งส่วนบุคคล', 'รถตู้', 'รถปิคอัพโดยสาร', 'รถโดยสารมากกว่า4ล้อ',
       'รถปิคอัพบรรทุก4ล้อ', 'รถบรรทุก6ล้อ', 'รถบรรทุกไม่เกิน10ล้อ',
       'รถบรรทุกมากกว่า10ล้อ', 'รถอีแต๋น', 'อื่นๆ', 'คนเดินเท้า',
       'จำนวนผู้เสียชีวิต', 'จำนวนผู้บาดเจ็บสาหัส', 'จำนวนผู้บาดเจ็บเล็ก',
       'รวมจำนวนผู้บาดเจ็บ']

province_json_path = 'thailand_provinces.json'

### แบบที่ 1 :
* Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และตัด row ที่มี missing value ออก
* Numerical : นำ Row ที่เป็น Outlier และ Missing Value ออก

In [7]:
columns_to_clean = [
    'จังหวัด', 'รถคันที่1', 'บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน', 'ลักษณะการเกิดอุบัติเ', 
    'สภาพอากาศ', 'จำนวนรถที่เกิดเหตุ', 'จำนวนที่เกิดเหตุทั้ง', 'รถจักรยานยนต์', 'รถสามล้อเครื่อง', 
    'รถยนต์นั่งส่วนบุคคล', 'รถตู้', 'รถปิคอัพโดยสาร', 'รถโดยสารมากกว่า4ล้อ', 'รถปิคอัพบรรทุก4ล้อ', 
    'รถบรรทุก6ล้อ', 'รถบรรทุกไม่เกิน10ล้อ', 'รถบรรทุกมากกว่า10ล้อ', 'รถอีแต๋น', 'อื่นๆ', 'คนเดินเท้า', 
    'จำนวนผู้เสียชีวิต', 'จำนวนผู้บาดเจ็บสาหัส', 'จำนวนผู้บาดเจ็บเล็ก', 'รวมจำนวนผู้บาดเจ็บ', 'มูลค่าความเสียหาย'
]

In [8]:
cleaned_1_df = df.dropna()
cleaned_1_df = remove_outliers(cleaned_1_df,numerical_feature)

cleaned_1_df = group_categorical_feature(cleaned_1_df)
cleaned_1_df = recalculate_accident_number(cleaned_1_df)

# Model Development
to_delete = ['LATITUDE', 'LONGITUDE','จังหวัด', 'รถคันที่1', 'บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน',
             'ลักษณะการเกิดอุบัติเ', 'สภาพอากาศ']
model_1 = cleaned_1_df.drop(to_delete,axis=1)
model_1 = encode_categorical_features(model_1,target_col = 'มูลค่าความเสียหาย')

# Scale Different Way
model_1_standard = scale_data(model_1,scale_by='standard_scaler')
model_1_minmax = scale_data(model_1,scale_by='min_max')

In [10]:
cleaned_1_df.to_csv('./output/data_after_clean_1.csv',encoding='utf-8-sig',index = False)

### แบบที่ 2 :
* Categorical Feature : group ข้อมูลเป็นกลุ่มใหญ่ ๆ และเติม Missing value ด้วย Mode
* Numerical : เติม Missing value ด้วย median และ cap ค่า outlier

In [9]:
cleaned_2_df = clean_by_cap_and_fill(df,numerical_feature,categorical_feature,province_json_path)

# Model Development
to_delete = ['LATITUDE', 'LONGITUDE','จังหวัด', 'รถคันที่1', 'บริเวณที่เกิดเหตุ', 'มูลเหตุสันนิษฐาน',
             'ลักษณะการเกิดอุบัติเ', 'สภาพอากาศ']
model_2 = cleaned_2_df.drop(to_delete,axis=1)
model_2 = encode_categorical_features(model_2,target_col = 'มูลค่าความเสียหาย')

# Scale Different Way
model_2_standard = scale_data(model_2,scale_by='standard_scaler')
model_2_minmax = scale_data(model_2,scale_by='min_max')

In [11]:
cleaned_2_df.to_csv('./output/data_after_clean_2.csv',encoding='utf-8-sig',index = False)

# ทด

In [ ]:
def yeo_johnson_transform(df, columns):
    df = df.copy()  # ป้องกันการเปลี่ยนแปลง DataFrame ต้นฉบับ
    pt = PowerTransformer(method='yeo-johnson')  # ใช้วิธี Yeo-Johnson

    for column in columns:
        if column in df.columns and pd.api.types.is_numeric_dtype(df[column]):
            non_na_values = df[column].dropna().values.reshape(-1, 1)  # เอาค่าที่ไม่ใช่ NaN
            if len(np.unique(non_na_values)) > 1:  # เช็คว่ามีค่าหลายค่า (ไม่นับ NaN)
                transformed_values = pt.fit_transform(non_na_values).flatten()
                df.loc[df[column].notna(), column] = transformed_values
    return df

In [ ]:
def log_transform(df, columns):
    df = df.copy()  # ป้องกันการเปลี่ยนแปลง DataFrame ต้นฉบับ
    for column in columns:
        if column in df.columns and pd.api.types.is_numeric_dtype(df[column]):
            df[column] = np.log(df[column])  # ใช้ log(1+x) เพื่อจัดการค่า 0
        else:
            print(f"Column '{column}' ไม่ใช่ตัวเลขหรือไม่มีอยู่ใน DataFrame")
    return df